# Картинки

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 1. Загрузка изображения
img = cv2.imread('image.jpg')  # BGR
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # конвертация в RGB

# 2. Просмотр
plt.imshow(img_rgb)
plt.axis('off')
plt.show()

# 3. Информация
print(f"Размер: {img.shape}")  # (высота, ширина, каналы)
print(f"Тип: {img.dtype}")
print(f"Диапазон: {img.min()}-{img.max()}")

# 4. Преобразования
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # черно-белое
resized = cv2.resize(img, (224, 224))  # изменение размера
blurred = cv2.GaussianBlur(img, (5, 5), 0)  # размытие

# 5. Нормализация для нейросетей
normalized = img / 255.0  # [0, 1]
standardized = (img - img.mean()) / img.std()  # mean=0, std=1

# 6. Кластеризация цветов (KMeans)
pixels = img_rgb.reshape(-1, 3)  # из (h,w,3) в (h*w, 3)
kmeans = KMeans(n_clusters=5, random_state=42)
labels = kmeans.fit_predict(pixels)
colors = kmeans.cluster_centers_

# 7. Восстановление сжатого изображения
compressed = colors[labels].reshape(img_rgb.shape).astype('uint8')

# 8. Сравнение
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].imshow(img_rgb); axes[0].set_title('Оригинал'); axes[0].axis('off')
axes[1].imshow(compressed); axes[1].set_title('5 цветов'); axes[1].axis('off')
plt.show()

# 9. PCA для уменьшения цветовых каналов
pca = PCA(n_components=3)  # оставляем 3 главных компоненты
pixels_pca = pca.fit_transform(pixels)
img_pca = pixels_pca.reshape(img_rgb.shape)

# 10. Гистограмма цветов
plt.figure(figsize=(10, 4))
colors = ('r', 'g', 'b')
for i, color in enumerate(colors):
    hist = cv2.calcHist([img_rgb], [i], None, [256], [0, 256])
    plt.plot(hist, color=color)
plt.title('Гистограмма цветов')
plt.show()

# 11. Сохранение
cv2.imwrite('compressed.jpg', cv2.cvtColor(compressed, cv2.COLOR_RGB2BGR))

# Краткая версия

In [ ]:
import cv2
import numpy as np

# Загрузка и базовая обработка
img = cv2.imread('image.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Для нейросетей
img_ready = cv2.resize(img_rgb, (224, 224)) / 255.0

# Кластеризация цветов
pixels = img_rgb.reshape(-1, 3)
df_pixels = pd.DataFrame(pixels, columns=['R', 'G', 'B'])
df_pixels['cluster'] = KMeans(n_clusters=5).fit_predict(pixels)

# Работа с папкой изображений:

In [ ]:
import os
from PIL import Image

# Загрузка всех изображений из папки
image_dir = 'images/'
images = []
for filename in os.listdir(image_dir)[:10]:  # первые 10
    if filename.endswith(('.jpg', '.png')):
        img = cv2.imread(os.path.join(image_dir, filename))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img_rgb, (100, 100))  # одинаковый размер
        images.append(img_resized)

# Создание датасета
X = np.array(images)  # (n_images, height, width, channels)
print(f"Датасет: {X.shape}")

# Для CNN (нейросети):

In [ ]:
# Подготовка для Keras/TensorFlow
def prepare_image_for_cnn(img_path, target_size=(224, 224)):
    img = cv2.imread(img_path)
    img = cv2.resize(img, target_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0  # нормализация
    return img

# Пакетная обработка
batch = np.array([prepare_image_for_cnn(path) for path in image_paths])
print(f"Batch shape: {batch.shape}")  # (batch_size, 224, 224, 3)

# Аугментация данных:

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Аугментация для увеличения данных
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# Пример аугментации
augmented = datagen.flow(batch, batch_size=32)

# 5 строках:

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

img = cv2.cvtColor(cv2.imread('image.jpg'), cv2.COLOR_BGR2RGB)
pixels = img.reshape(-1, 3)
colors = KMeans(n_clusters=5).fit(pixels).cluster_centers_
labels = KMeans(n_clusters=5).fit_predict(pixels)
compressed = colors[labels].reshape(img.shape).astype('uint8')

cv2.imread() → BGR формат

cv2.cvtColor(..., COLOR_BGR2RGB) → для matplotlib

/255.0 → нормализация для нейросетей

reshape(-1, 3) → для работы с пикселями

KMeans на пикселях → кластеризация цветов



# 4. ОБРАБОТКА КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ

In [ ]:
# Target Encoding (очень важно!)
from category_encoders import TargetEncoder
encoder = TargetEncoder()
df['cat_encoded'] = encoder.fit_transform(df['category'], df['target'])

In [ ]:
# Быстрая проверка всех моделей
from sklearn.model_selection import cross_val_score
models = [LogisticRegression(), RandomForestClassifier(), XGBClassifier()]
for model in models:
    score = cross_val_score(model, X, y, cv=3, scoring='f1').mean()
    print(f"{model.__class__.__name__}: {score:.4f}")